In [ ]:
%pip install torch torch-geometric
%pip install pandas

: 

In [15]:
import pandas as pd

genuine_users = pd.read_csv('./cresci-2017/genuine_accounts/users.csv', encoding_errors='replace')
genuine_tweets = pd.read_csv('cresci-2017/genuine_accounts/tweets.csv', encoding_errors='replace')
social_spam_users_1 = pd.read_csv('cresci-2017/social_spambots_1/users.csv', encoding_errors='replace')
social_spam_tweets_1 = pd.read_csv('cresci-2017/social_spambots_1/tweets.csv', encoding_errors='replace')
social_spam_users_2 = pd.read_csv('cresci-2017/social_spambots_2/users.csv', encoding_errors='replace')
social_spam_tweets_2 = pd.read_csv('cresci-2017/social_spambots_2/tweets.csv', encoding_errors='replace')
social_spam_users_3 = pd.read_csv('cresci-2017/social_spambots_3/users.csv', encoding_errors='replace')
social_spam_tweets_3 = pd.read_csv('cresci-2017/social_spambots_3/tweets.csv', encoding_errors='replace')
traditional_spam_users = pd.read_csv('cresci-2017/traditional_spambots_1/users.csv', encoding_errors='replace')
traditional_spam_tweets = pd.read_csv('cresci-2017/traditional_spambots_1/tweets.csv', encoding_errors='replace')
# fake_follower_users = pd.read_csv('cresci-2017/fake_followers/users.csv', encoding_errors='replace')
# fake_follower_tweets = pd.read_csv('cresci-2017/fake_followers/tweets.csv', encoding_errors='replace')

# # example of getting only reply tweets
# filtered_genuine_tweeets = genuine_tweets[genuine_tweets['in_reply_to_user_id'] != 0.0]

print(genuine_users.columns)
print(genuine_tweets.columns)
print(social_spam_users_1.columns)
print(social_spam_tweets_1.columns)
print(social_spam_users_2.columns)
print(social_spam_tweets_2.columns)
print(social_spam_users_3.columns)
print(social_spam_tweets_3.columns)
print(traditional_spam_users.columns)
print(traditional_spam_tweets.columns)

c:\Users\work\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\work\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\work\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'time_zone', 'location', 'default_profile', 'default_profile_image',
       'geo_enabled', 'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'is_translator', 'follow_request_sent', 'protected', 'verified',
       'notifications', 'description', 'contributors_enabled', 'following',
       'created_at', 'timestamp', 'crawled_at', 'updated', 'test_set_1',
       'test_set_2'],
      dtype='object')
Index(['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_user_id

In [ ]:
# Combining data into a single dataframe
genuine_users['label'] = 0
social_spam_users_1['label'] = 1
social_spam_users_2['label'] = 1
social_spam_users_3['label'] = 1
traditional_spam_users['label'] = 1

all_users = pd.concat([
    genuine_users,
    social_spam_users_1,
    social_spam_users_2,
    social_spam_users_3,
    traditional_spam_users
], ignore_index=True)

# all_users.head()

In [ ]:
# Preparing relevant features
import numpy as np

features = [
    'statuses_count', 'followers_count', 'friends_count',
    'favourites_count', 'listed_count', 'default_profile',
    'default_profile_image', 'geo_enabled', 'protected', 'verified'
]

for col in features:
    if all_users[col].dtype == bool:
        all_users[col] = all_users[col].astype(int)

X = all_users[features].fillna(0).to_numpy(dtype=np.float32)
y = all_users['label'].to_numpy(dtype=np.int64)


In [20]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import numpy as np

# Sample graph with 5 nodes
edge_index = torch.tensor([[0, 1, 2, 3, 4, 1],
                           [1, 0, 3, 2, 1, 4]], dtype=torch.long)  # directed

# Node features: tweet frequency, URL presence, hashtag density, etc.
x = torch.tensor([
    [0.7, 1.0, 0.5, 0.8],  # user 0
    [0.2, 0.0, 0.1, 0.4],  # user 1
    [0.9, 1.0, 0.6, 0.7],  # user 2
    [0.1, 0.0, 0.2, 0.3],  # user 3
    [0.8, 1.0, 0.4, 0.9],  # user 4
], dtype=torch.float)

# Labels: 1 = bot, 0 = human
y = torch.tensor([1, 0, 1, 0, 1], dtype=torch.long)

# Use a basic graph object
data = Data(x=x, edge_index=edge_index, y=y)


GNN model


In [46]:
from torch.nn import Linear

class BotClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.lin(x)
        return x


Reinfrocement learning

In [ ]:
model = BotClassifier(input_dim=4, hidden_dim=8, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def compute_reward(pred, y_true):
    return (pred.argmax(dim=1) == y_true).float() * 1.0 - 0.5  # +1 or -0.5

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    out = model(data)
    loss = F.cross_entropy(out, data.y)

    # Simulate "reward" from environment
    reward = compute_reward(out, data.y).mean()

    # RL-inspired update: reward modulates loss
    total_loss = loss - 0.01 * reward  # encourage higher reward
    total_loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        acc = (out.argmax(dim=1) == data.y).float().mean().item()
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Acc: {acc:.2f}, Reward: {reward.item():.2f}")


Epoch 0, Loss: 0.6980, Acc: 0.40, Reward: -0.10
Epoch 10, Loss: 0.6765, Acc: 0.60, Reward: 0.10
Epoch 20, Loss: 0.6749, Acc: 0.60, Reward: 0.10
Epoch 30, Loss: 0.6732, Acc: 0.60, Reward: 0.10
Epoch 40, Loss: 0.6713, Acc: 0.60, Reward: 0.10
Epoch 50, Loss: 0.6695, Acc: 0.60, Reward: 0.10
Epoch 60, Loss: 0.6658, Acc: 0.60, Reward: 0.10
Epoch 70, Loss: 0.6575, Acc: 0.60, Reward: 0.10
Epoch 80, Loss: 0.6416, Acc: 0.60, Reward: 0.10
Epoch 90, Loss: 0.6155, Acc: 0.60, Reward: 0.10
Epoch 100, Loss: 0.5779, Acc: 0.60, Reward: 0.10
Epoch 110, Loss: 0.5325, Acc: 0.80, Reward: 0.30
Epoch 120, Loss: 0.4833, Acc: 0.80, Reward: 0.30
Epoch 130, Loss: 0.4376, Acc: 0.80, Reward: 0.30
Epoch 140, Loss: 0.3986, Acc: 0.80, Reward: 0.30
Epoch 150, Loss: 0.3676, Acc: 0.80, Reward: 0.30
Epoch 160, Loss: 0.3448, Acc: 0.80, Reward: 0.30
Epoch 170, Loss: 0.3283, Acc: 0.80, Reward: 0.30
Epoch 180, Loss: 0.3169, Acc: 0.80, Reward: 0.30
Epoch 190, Loss: 0.3099, Acc: 0.80, Reward: 0.30
Epoch 200, Loss: 0.3038, Acc: 

In [48]:
model.eval()
preds = model(data).argmax(dim=1)
print("Predicted:", preds.tolist())
print("Actual:   ", data.y.tolist())


Predicted: [1, 0, 1, 1, 1]
Actual:    [1, 0, 1, 0, 1]
